<a href="https://colab.research.google.com/github/Sloba98/Laundry-Image-Classification/blob/restnet_fine_tuning/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import torch
from torch import nn
from torch import optim
from torchvision import models, transforms
from torchvision.models import ResNet50_Weights
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import functional
from sklearn.metrics import confusion_matrix
import numpy as np
import torchvision

In [13]:
from PIL import Image
from torchvision import transforms

In [14]:

from matplotlib import pyplot as plt
from matplotlib import image as mpimg


In [15]:
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/laundry classification/Laundry-Image-Classification/inputs/test', transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [55]:
def load_model(model_path, device):
    model = torchvision.models.resnet50()
    model.fc = nn.Linear(model.fc.in_features, 10)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

In [56]:
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_accuracy = running_corrects.double() / total

    confusion_mat = confusion_matrix(all_labels, all_preds)

    return epoch_loss, epoch_accuracy.item(), confusion_mat


In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [58]:
model_paths = {"base": "/content/resnet_base_32",
          "augmentation": "/content/resnet_base_32_augmentation",
          "finetunning": "/content/resnet_base_32_early_stop_001_lr"}
criterion = nn.CrossEntropyLoss()


In [66]:
model = load_model(model_paths['base'], device=device)
loss, accuracy, confusion_mat = evaluate_model(model=model, dataloader=test_loader, criterion=criterion, device=device)

<ipython-input-55-0957ae11bfc0>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [67]:
print(f"Loss: {loss}, Accuracy: {accuracy}")
print(confusion_mat)

Loss: 0.4596870057685401, Accuracy: 0.8387096774193549
[[12  0  0  0  0  1  0  2  0  0]
 [ 0 10  0  1  0  0  1  0  0  0]
 [ 1  1 53  4  0  4  0  1  2  6]
 [ 0  0  4 31  0  2  0  0  1  0]
 [ 0  0  0  0 39  0  0  2  1  0]
 [ 0  0  1  2  0 21  0  1  0  1]
 [ 0  1  0  0  0  1 71  0  0  0]
 [ 0  0  0  0  1  0  0 29  0  0]
 [ 1  1  0  0  0  0  0  2  8  0]
 [ 3  0  6  0  0  5  0  0  0 38]]


In [68]:
model = load_model(model_paths['augmentation'], device=device)
loss, accuracy, confusion_mat = evaluate_model(model=model, dataloader=test_loader, criterion=criterion, device=device)

<ipython-input-55-0957ae11bfc0>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [69]:
print(f"Loss: {loss}, Accuracy: {accuracy}")
print(confusion_mat)

Loss: 0.4803991975483074, Accuracy: 0.8413978494623656
[[13  0  0  0  0  1  0  1  0  0]
 [ 0  9  0  1  0  0  2  0  0  0]
 [ 3  2 45  8  0  5  0  0  1  8]
 [ 0  0  1 35  0  1  0  0  1  0]
 [ 0  0  0  0 42  0  0  0  0  0]
 [ 1  0  0  8  0 14  0  1  0  2]
 [ 0  0  0  0  0  0 73  0  0  0]
 [ 0  0  0  1  2  0  0 27  0  0]
 [ 1  0  0  1  0  0  0  2  8  0]
 [ 1  0  1  0  0  2  0  1  0 47]]


In [70]:
model = load_model(model_paths['finetunning'], device=device)
loss, accuracy, confusion_mat = evaluate_model(model=model, dataloader=test_loader, criterion=criterion, device=device)

<ipython-input-55-0957ae11bfc0>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [71]:
print(f"Loss: {loss}, Accuracy: {accuracy}")
print(confusion_mat)

Loss: 0.5087916924748369, Accuracy: 0.8575268817204301
[[11  0  1  0  0  1  0  0  2  0]
 [ 1  2  1  0  0  0  6  0  2  0]
 [ 1  0 65  0  0  4  0  0  0  2]
 [ 0  0  7 24  0  7  0  0  0  0]
 [ 0  0  0  0 42  0  0  0  0  0]
 [ 0  0  1  0  0 24  0  0  0  1]
 [ 0  1  0  0  0  1 71  0  0  0]
 [ 0  0  0  0  3  0  0 26  1  0]
 [ 0  0  0  1  0  0  1  0 10  0]
 [ 2  0  5  0  0  1  0  0  0 44]]
